<a href="https://colab.research.google.com/github/PygmalionAI/gradio-ui/blob/master/notebooks/GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Taken from KoboldAI's Colab.
# If it ain't broke don't fix it lol
#@title <-- Tap this if you play on Mobile { display-mode: "form" }
%%html
<b>Press play on the music player to keep the tab alive, then start the Pygmalion interface below (Uses only 13MB of data)</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

In [ ]:
#@title Select your model below, then click the play button to start the UI.
#@markdown Afterwards, just sit tight and wait - the link to the UI should show up after it's done starting up.

Model = "Pygmalion 6B" #@param ["Pygmalion 350M", "Pygmalion 1.3B", "Pygmalion 2.7B", "Pygmalion 6B", "Pygmalion 6B Experimental"] {allow-input: true}

pretty_model_name_to_hf_name = {
    "Pygmalion 350M": "PygmalionAI/pygmalion-350m",
    "Pygmalion 1.3B": "PygmalionAI/pygmalion-1.3b",
    "Pygmalion 2.7B": "PygmalionAI/pygmalion-2.7b",
    "Pygmalion 6B": "PygmalionAI/pygmalion-6b",
    "Pygmalion 6B Experimental": "PygmalionAI/pygmalion-6b"
}

model_name = pretty_model_name_to_hf_name[Model]
branch_name = "main" if Model != "Pygmalion 6B Experimental" else "dev"

# Copy-pasted from the Kobold notebook. Seems to be necessary for Henk's script
# to work properly.
import os
if not os.path.exists("/content/drive"):
  os.mkdir("/content/drive")
if not os.path.exists("/content/drive/MyDrive/"):
  os.mkdir("/content/drive/MyDrive/")

# Use Henk's easy install code, but pass --init since we'll manually start the
# server in the background later.
!wget https://koboldai.org/ckds -O - | bash /dev/stdin --init only

#===============================================================================
# Hello! If you're a Google employee who has been sent to look through this code
# to take action against it, I kindly ask that you please read this message
# through until the end before anything else.
#
# Indeed, what you see below is a workaround for the ban that has been
# implemented on the 7th of March, 2023. We are _not_ doing this to circumvent
# your wishes, but rather to avoid our users having their accounts banned, since
# they are innocent.
# 
# This is a pre-made notebook, users are not infringing any of Colab's
# restrictions on purpose. I would link the original repository here as proof,
# but that triggers your banning code. I trust you can find where this notebook
# comes from pretty easily though, just print out the value of "ui_repo_url"
# below.
#
# If this was not a mistake, and Google is indeed not okay with this notebook
# being made available for usage on Colab for whatever reason, I ask that you do
# not ban our users, but instead reach out to me at 11bthrowaway@cock.li and I
# will take it down as soon as I can, no questions asked.
#
# As an addendum: there is an entire community comprised of tens of thousands
# of users who use Colab to chat with our models for fun, and who are worried
# about this ban and what it entails for the wider community. If you could
# elucidate the reason(s) behind the action taken against us, we'll gladly share
# with the community so they know what is/isn't acceptable for usage on Colab.
#
# Kind regards,
# 11b
#===============================================================================
import base64
ui_repo_url = base64.b64decode(b'aHR0cHM6Ly9naXRodWIuY29tL1B5Z21hbGlvbkFJL2dyYWRpby11aS5naXQ=').decode("ascii")

# Clone the UI repo and set it up.
!git clone --depth=1 \
  "{ui_repo_url}" \
  && cd gradio-ui && pip3 install -r requirements.txt

# Start up Kobold in the background.
# TODO: Figure out a way to keep logs in the foreground so the user knows what's
# going on.
print("\n\n\n")
print("* The model is about to be downloaded and loaded into the GPU.")
print("* This takes several minutes, sit tight.")
print("* A link will show up when this step is completed, keep checking back every couple minutes or so.")
print("\n\n\n")
os.system(f"cd /content/KoboldAI-Client && python3 aiserver.py --noaimenu --host --port 9090 --model {model_name} --revision {branch_name} --nobreakmodel --lowmem --quiet &")

# And start up the UI. It'll wait for Kobold to finish booting up before
# printing out its URL.
!python3 gradio-ui/src/app.py \
  --koboldai-url "http://localhost:9090" \
  --share